# 웹 크롤링

In [1]:
#!pip install selenium
#!pip install webdriver_manager

In [2]:
import string
import time, re
import pandas as pd
import requests
import sys
from tqdm import tqdm
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import warnings
warnings.filterwarnings('ignore')

In [3]:
options = webdriver.ChromeOptions() 
options.add_argument("--start-maximized");  # Chrome 브라우저 최대화 설정
#options.add_argument('--headless')  # headless 모드
options.add_argument('--disable-gpu')  # 그래픽 가속 해제 (오류 방지)
options.add_argument('--mute-audio')  # 음소거모드 적용
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [4]:
cols = ['Exhibitor', 'Link']

# 링크부터 크롤링

In [ ]:
company_df = pd.DataFrame(columns = cols)

NEWS_MAX_CNT = 24

In [ ]:
exhibitors_df = pd.DataFrame(columns = cols)

NEWS_MAX_CNT = 24
try :
    for pages in string.ascii_uppercase :
        driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

        past_li = []
        page_cnt = 1

        while True :
            li = []
            
            driver.get(f"https://www.mwcbarcelona.com/exhibitors?letter={pages}&page={page_cnt}")
            time.sleep(1)
            try : 
                driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
            except :
                pass
            
            names = driver.find_elements(By.CLASS_NAME, 'ais-Highlight-nonHighlighted')
            links = driver.find_elements(By.CSS_SELECTOR, 'tr[role="link"]')

            for name, link in zip(names,links) :
                if len(name.text) != 0 :
                    li.append([name.text, link.get_attribute('data-href')])

            if past_li == li :
                break

            temp_df = pd.DataFrame(li, columns = cols)

            exhibitors_df = pd.concat([exhibitors_df, temp_df], ignore_index = True)

            if len(li) < NEWS_MAX_CNT :
                break

            past_li = li
            page_cnt += 1

        driver.quit()
except :
    print('check')
    driver.quit()
    pass

    exhibitors_df.drop_duplicates(['Exhibitor'], keep = 'first', inplace = True)

    
exhibitors_df

In [ ]:
exhibitors_df.to_csv('crawling.csv', encoding='utf-8-sig')

In [ ]:
exhibitors_df

In [6]:
import pandas as pd

exhibitors_df = pd.read_csv('crawling.csv')
exhibitors_df

,Unnamed: 0,Exhibitor,Link
0,0,ALAT,/exhibitors/28808-alat
1,1,A-Champs Interactive Training Solutions,/exhibitors/28761-a-champs-interactive-trainin...
2,2,A-INFO Inc,/exhibitors/28178-a-info-inc
3,3,AALTO HAPS Limited,/exhibitors/26673-aalto-haps-limited
4,4,Aava Mobile Oy - a Pepperl+Fuchs company,/exhibitors/27513-aava-mobile-oy-a-pepperl-fuc...
...,...,...,...
2401,2401,Zoundream SL,/exhibitors/27574-zoundream-sl
2402,2402,ZTE Corporation,/exhibitors/26781-zte-corporation
2403,2403,ZTT GROUP,/exhibitors/27027-ztt-group
2404,2404,Zubilabs,/exhibitors/28629-zubilabs


In [ ]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
time.sleep(1)

driver.get("https://www.mwcbarcelona.com/" + exhibitors_df['Link'][3])
time.sleep(1)

driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
time.sleep(1)

web = driver.find_element(By.NAME, "flex w-full md:w-1/3 pr-2")
print(web.get_attribute('href'))

In [ ]:
location = driver.find_elements(By.CSS_SELECTOR, '#exhibitor-container > aside > div:nth-child(1) > ul > a').text

interest = []
interests = driver.find_elements(By.CSS_SELECTOR, '.w-full.pb-4.mb-4.border-b.border-gray-300 ul.flex.flex-wrap li')
for i in interests :
    interest.append(i)

In [43]:
url

'https://www.mwcbarcelona.com//exhibitors/28808-alat'

In [49]:
url = "https://www.mwcbarcelona.com/" + exhibitors_df['Link'][3]
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [50]:
elements = soup.select('#exhibitor-container > aside > div:nth-child(1) > ul > a')

location_list = [element.text.strip() for element in elements]
location = ''.join(location_list)
location

'Hall 4 Stand 4B25'

In [51]:
elements = soup.select('#exhibitor-container > aside > div:nth-child(3) > ul > li')

interest_list = [element.text.strip() for element in elements]
interest_text = ', '.join(interest_list)
interest_text

'SYSTEMS INTEGRATION, NETWORK INFRASTRUCTURE, SATELLITE/NON TERRESTRIAL NETWORKS, CLIMATE/ENVIRONMENT/SUSTAINABILITY, MNO/MVNO'